In [ ]:
import numpy as np
import os
from ase import Atoms
from ase.io import read, write
from ase.build import molecule
from ase.constraints import FixAtoms

# --- Matlantis Features & Utils ---
try:
    from matlantis_features.features.md import (
        MDFeature, ASEMDSystem, MDExtensionBase, NVTBerendsenIntegrator
    )
    from matlantis_features.utils.calculators import pfp_estimator_fn
    # ご自身のUtilsがある場合
    from lib2_utils.md_utils import PrintWriteLog 
except ImportError:
    # Utilsがない場合のフォールバック (簡易版PrintWriteLog)
    from time import perf_counter
    class PrintWriteLog(MDExtensionBase):
        def __init__(self, fname, dirout='.', stdout=False):
            self.fname, self.dirout, self.stdout = fname, dirout, stdout
        def __call__(self, system, integrator):
            pass # 省略

# -------------------------------------------------------------------------
# 1. 構造構築関数: Al2O3抽出 + HF配置
# -------------------------------------------------------------------------

def prepare_al2o3_with_hf(source_xyz, n_hf=30, vacuum_height=20.0):
    """
    PVDF/Al2O3構造からAl2O3のみを抽出し、真空層にHFを配置する
    """
    print(f"--- 構造構築開始 ---")
    # 1. 読み込み
    if not os.path.exists(source_xyz):
        raise FileNotFoundError(f"{source_xyz} が見つかりません")
    
    full_structure = read(source_xyz)
    
    # 2. AlとOのみ抽出 (PVDFのC,H,Fを除去)
    # リスト内包表記で Al, O のインデックスを取得
    indices = [a.index for a in full_structure if a.symbol in ['Al', 'O']]
    al2o3_slab = full_structure[indices]
    
    print(f"  抽出された原子数: {len(al2o3_slab)} (Al, Oのみ)")
    
    # 3. 制約（FixAtoms）の解除
    # 表面反応を見たいので、もし固定されていても一旦解除、あるいは最下層のみ固定等の調整
    # ここでは簡単のため、最下層の数Åだけ固定しなおす例を示します
    al2o3_slab.set_constraint() # 全解除
    z_positions = al2o3_slab.positions[:, 2]
    z_min = z_positions.min()
    # 底面から3Å以内の原子を固定
    fixed_indices = [a.index for a in al2o3_slab if a.position[2] < z_min + 3.0]
    al2o3_slab.set_constraint(FixAtoms(indices=fixed_indices))
    
    # 4. セル拡張 (真空層の追加)
    # 現在のZ最大値を取得
    z_max = z_positions.max()
    cell = al2o3_slab.get_cell()
    
    # 新しいZ軸の長さ = (現在の表面位置) + (真空層)
    new_z_height = z_max + vacuum_height
    cell[2, 2] = new_z_height
    al2o3_slab.set_cell(cell)
    al2o3_slab.set_pbc(True)
    
    # 5. HF分子の配置
    print(f"  HF分子 {n_hf} 個を表面上に配置します...")
    hf_template = molecule('HF')
    
    # 配置エリア: 表面から 2.5Å 〜 10Å の間
    z_range_min = z_max + 2.5
    z_range_max = z_max + 10.0
    
    # XY平面の範囲
    x_len = cell[0, 0] # 直交セルと仮定
    y_len = cell[1, 1] # 直交セルと仮定 (非直交の場合は要調整)
    if x_len < 1.0: # ベクトル形式の場合
        x_len = np.linalg.norm(cell[0])
        y_len = np.linalg.norm(cell[1])

    for i in range(n_hf):
        hf = hf_template.copy()
        
        # ランダムな位置 (x, y, z)
        rand_x = np.random.rand() * x_len
        rand_y = np.random.rand() * y_len
        rand_z = z_range_min + np.random.rand() * (z_range_max - z_range_min)
        
        # ランダムな向き
        hf.rotate(np.random.rand()*360, 'x')
        hf.rotate(np.random.rand()*360, 'y')
        
        # 配置
        hf.translate([rand_x, rand_y, rand_z])
        
        # はみ出しチェック (PBC) は extend 時に自動ではないため、
        # 原点付近に移動してから配置するか、追加後に wrap() する
        al2o3_slab.extend(hf)
        
    return al2o3_slab

# -------------------------------------------------------------------------
# 2. エッチング反応追跡ロガー (TrackEtching)
# -------------------------------------------------------------------------

class TrackEtching(MDExtensionBase):
    """
    Al-F結合（エッチング）と O-H結合（水生成の前駆）を追跡するクラス
    """
    def __init__(self, fname, dirout='.', stdout=True):
        self.log_path = f'{dirout}/{fname}_etching.log'
        self.stdout = stdout

    def __call__(self, system, integrator):
        atoms = system.ase_atoms
        n_step = system.current_total_step
        time_ps = system.current_total_time / 1000.0
        
        # ヘッダー
        if n_step == 0:
            with open(self.log_path, 'w') as f:
                f.write("step,time_ps,n_AlF_bonds,n_OH_bonds,n_H2O_mols\n")

        # インデックス取得
        al_idx = [a.index for a in atoms if a.symbol == 'Al']
        o_idx = [a.index for a in atoms if a.symbol == 'O']
        f_idx = [a.index for a in atoms if a.symbol == 'F']
        h_idx = [a.index for a in atoms if a.symbol == 'H']
        
        n_alf = 0
        n_oh = 0
        n_h2o = 0
        
        try:
            # 距離行列一括取得 (安全策)
            dists = atoms.get_all_distances(mic=True)
            
            # 1. Al-F 結合数 (エッチングの指標) < 2.0 A
            if al_idx and f_idx:
                alf_sub = dists[np.ix_(al_idx, f_idx)]
                n_alf = (alf_sub < 2.0).sum()
                
            # 2. O-H 結合数 (表面水酸基化の指標) < 1.1 A
            if o_idx and h_idx:
                oh_sub = dists[np.ix_(o_idx, h_idx)]
                n_oh = (oh_sub < 1.1).sum()
                
                # (簡易的) H2O分子数推定: 1つのOに対して2つのHが結合している数
                # O原子ごとにループして確認
                oh_bool = (oh_sub < 1.1) # (n_O, n_H) のTrue/False行列
                h_per_o = oh_bool.sum(axis=1) # 各O原子に結合しているHの数
                n_h2o = (h_per_o == 2).sum()

        except Exception as e:
            print(f"Etching Analysis Error: {e}")

        line = f"{n_step},{time_ps:.2f},{n_alf},{n_oh},{n_h2o}"
        with open(self.log_path, 'a') as f:
            f.write(line + "\n")
        
        if self.stdout:
            print(f"ETCH LOG: {line}")

# -------------------------------------------------------------------------
# 3. メイン実行部
# -------------------------------------------------------------------------

if __name__ == "__main__":
    
    # 設定
    source_file = "/home/jovyan/Kaori/MD/LiB_2/structure/output/PVDF_interfaces/PVDF_on_Al2O3_cell_repeat_2x2x1_final.xyz"
    output_dir = "validation_etching"
    os.makedirs(output_dir, exist_ok=True)
    
    # PFP設定
    model_version = 'v7.0.0'
    calc_mode = 'CRYSTAL_U0'
    
    # 1. システム構築
    try:
        # HFを多め(50個)に入れて反応確率を上げる
        atoms_init = prepare_al2o3_with_hf(source_file, n_hf=50, vacuum_height=20.0)
        atoms_init.write(f"{output_dir}/init_system.cif")
        print("初期構造を保存しました。")
    except Exception as e:
        print(f"構造構築エラー: {e}")
        exit()

    # 2. MD実行ループ (350K と 600K)
    temps = [350.0, 600.0]
    
    for T in temps:
        print(f"\n=== エッチング検証 MD: {T} K ===")
        
        fname = f"etching_test_{int(T)}K"
        
        # Estimator
        estimator = pfp_estimator_fn(model_version=model_version, calc_mode=calc_mode)
        
        # System
        system = ASEMDSystem(atoms_init.copy(), step=0, time=0.0)
        
        # Integrator (NVT)
        integrator = NVTBerendsenIntegrator(timestep=0.5, temperature=T, taut=100.0)
        
        # Feature
        md = MDFeature(
            integrator=integrator,
            n_run=40000, # 20 ps (0.5fs * 40000)
            traj_file_name=f"{output_dir}/{fname}.traj",
            traj_freq=200,
            estimator_fn=estimator,
            logger_interval=200,
            show_logger=False
        )
        
        # Extensions
        logger_std = PrintWriteLog(fname, dirout=output_dir, stdout=True)
        logger_etch = TrackEtching(fname, dirout=output_dir, stdout=True)
        
        # Run
        md(system, extensions=[(logger_std, 200), (logger_etch, 200)])
        
        print(f"完了: {fname}")
        print("確認事項: n_AlF_bonds が増加しているか？ n_H2O_mols が生成されているか？")

--- 構造構築開始 ---
  抽出された原子数: 1520 (Al, Oのみ)
  HF分子 50 個を表面上に配置します...
初期構造を保存しました。

=== エッチング検証 MD: 350.0 K ===


/home/jovyan/.py313/lib/python3.13/site-packages/ase/io/cif.py:836: UserWarning: Occupancies present but no occupancy info for "{symbol}"
  warnings.warn(str(err))
The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/validation_etching/etching_test_350K.traj.
Note: The max disk size of /home/jovyan is about 98G.


ETCH LOG: 0,0.00,0,0,0


/home/jovyan/.py313/lib/python3.13/site-packages/ase/md/nvtberendsen.py:98: RuntimeWarning: divide by zero encountered in scalar divide
  1.0 + (self.temperature / old_temperature - 1.0) * tautscl


ETCH LOG: 200,0.10,0,0,0
ETCH LOG: 400,0.20,3,3,0
ETCH LOG: 600,0.30,5,4,0
ETCH LOG: 800,0.40,6,4,0
ETCH LOG: 1000,0.50,7,4,0
ETCH LOG: 1200,0.60,7,3,0
ETCH LOG: 1400,0.70,8,5,0
ETCH LOG: 1600,0.80,10,5,0
ETCH LOG: 1800,0.90,10,4,0
ETCH LOG: 2000,1.00,15,7,0
